## Heston Calibration

In [ ]:
import pandas as pd
import numpy as np
from numba import jit, njit, prange 
from numba import cuda
from scipy.optimize import minimize
from kernel.market_data.market import Market
from plotly import graph_objects as go
from typing import Optional, Union
from collections.abc import Sequence

In [ ]:
from kernel.tools import CalendarConvention, ObservationFrequency, Model
from kernel.market_data import InterpolationType, VolatilitySurfaceType, Market, RateCurveType
from data.option_data import *

# 1. Contexte du Modèle Heston

Dans le modèle de Heston, la variance $v_t$ du sous-jacent évolue stochastiquement selon la dynamique suivante :

- **$\kappa$** : vitesse de réversion vers la moyenne,
- **$\theta$** : niveau moyen de variance à long terme,
- **$\sigma$** : volatilité de la variance (souvent appelée *vol-of-vol*),
- **$\rho$** : corrélation entre le mouvement du prix du sous-jacent et celui de sa variance,
- **$v_0$** : variance initiale.


La formule semi-analytique du call européen dans le modèle Heston se présente souvent sous la forme :

$$
C(S_t, K, T) = S_t \, P_1 \;-\; K\, e^{-rT}\, P_2,
$$

où les termes $P_1$ et $P_2$ (parfois notés aussi $\mathbb{Q}^1$ et $\mathbb{Q}^2$) se calculent à l’aide d’intégrales faisant intervenir la fonction caractéristique de la log-densité du sous-jacent sous deux mesures risques neutres distinctes.  
Explicitement :

$$
P_j \;=\; \frac{1}{2} \;+\; \frac{1}{\pi}\int_{0}^{+\infty} \Re\!\left(\frac{e^{-i\,s\,\ln K}\, f_j(s)}{i\,s}\right)\, ds,\quad j=1,2.
$$

---

# 2. Construction de la Fonction Caractéristique

Le point crucial de la méthode réside dans la construction d’une **fonction caractéristique complexe** adaptée au modèle Heston.

## 2.1. Définition générale de $f_j(s)$

Chaque intégrale fait intervenir une fonction $f_j$. On peut écrire (pour $j=1,2$) :

$$
f_j(s) \;=\; \exp\Bigl( C_j(s) \;+\; D_j(s)\,v_0 \;+\; i\,s\,x \Bigr),
$$

où $x = \ln S_t$. Les fonctions $C_j$ et $D_j$ dépendent des paramètres $\kappa$, $\theta$, $\sigma$, $\rho$ et de la mesure choisie (d'où l'indice $j$).

### 2.1.1. Paramètres auxiliaires

Pour distinguer les deux intégrales (liées à $P_1$ et $P_2$), on introduit parfois des coefficients tels que :

- $u_1 = 0.5,\quad u_2 = -0.5,$
- $b_1 = \kappa \;-\; \rho\,\sigma\,i\,s ,\quad b_2 = \kappa,$
  
On définit alors :

$$
d_j \;=\; \sqrt{\left(\rho\,\sigma\,i\,s - b_j\right)^{2} \;+\; \sigma^2\,\left(s^2 + i\,s\,(2\,u_j)\right)},
$$

et

$$
g_j \;=\; \frac{b_j \;-\; \rho\,\sigma\,i\,s - d_j}{b_j \;-\; \rho\,\sigma\,i\,s + d_j}.
$$

### 2.1.2. Fonctions $C_j$ et $D_j$

On peut alors exprimer :

$$
C_j(s) \;=\; r\,i\,s\,T \;+\; \frac{\kappa\,\theta}{\sigma^2}\,\Biggl[\left(b_j - \rho\,\sigma\,i\,s\right)T \;-\; 2\,\ln\!\left(\frac{1 - g_j\,e^{-d_j\,T}}{1 - g_j}\right)\Biggr],
$$

et

$$
D_j(s) \;=\; \frac{b_j - \rho\,\sigma\,i\,s + d_j}{\sigma^2}\,\left(\frac{1 - e^{-d_j\,T}}{1 - g_j\,e^{-d_j\,T}}\right).
$$

La fonction caractéristique s'obtient finalement par :

$$
f_j(s) \;=\; \exp\!\Bigl( C_j(s) + D_j(s)\,v_0 + i\,s\,x\Bigr).
$$

---

## 2.2. Version « unique » : $f_{\text{Heston}}(s)$

Plutôt que de séparer en deux fonctions $f_1(s)$ et $f_2(s)$, il est possible de regrouper les effets dans une **unique** fonction caractéristique en procédant comme suit :

1. **Calcul du paramètre $d$** :

   $$
   d \;=\; \sqrt{ \left( \rho\, \sigma\, i\, s - \kappa \right)^2 + \sigma^2 \left(i\, s + s^2\right) }.
   $$

2. **Définition du paramètre $g$** :

   $$
   g \;=\; \frac{\kappa - \rho\, \sigma\, i\, s + d}{\kappa - \rho\, \sigma\, i\, s - d}.
   $$

3. **Construction de la fonction caractéristique** en deux parties exponentielles :

   - Le premier terme :

     $$
     \text{Exp}_1 \;=\; \exp\!\Bigl(i\, s \,(\ln S_t + r\,T)\Bigr)\,\left(\frac{1 - g\,e^{-d\,T}}{1 - g}\right)^{-\frac{2\,\theta\,\kappa}{\sigma^2}}.
     $$

   - Le second terme :

     Avant de le formuler, on définit :

     $$
     g_1 \;=\; \kappa - \rho\, \sigma\, i\, s - d.
     $$

     Puis,

     $$
     \text{Exp}_2 \;=\; \exp\!\left(\frac{\theta\,\kappa\,T}{\sigma^2}\, g_1 \;+\; \frac{v_0}{\sigma^2}\,g_1\,\frac{1 - e^{-d\,T}}{1 - g\,e^{-d\,T}}\right).
     $$

Ainsi, la fonction caractéristique globale s'écrit :

$$
f_{\text{Heston}}(s) \;=\; \text{Exp}_1 \times \text{Exp}_2.
$$

---

# 3. Formulation Intégrale pour le Prix de l’Option

## 3.1. Expression du Prix en Version « Unique »

Une fois la fonction caractéristique définie, on regroupe les contributions pour obtenir le prix de l’option par :

$$
N(s) \;=\; f_{\text{Heston}}(s - i) - K \, f_{\text{Heston}}(s),
$$

et

$$
C(S_t,K,T) \;=\; \frac{1}{2}\left(S_t - K\,e^{-rT}\right) + \frac{1}{\pi}\,\int_{0}^{+\infty} \frac{N(s)}{e^{i\,s\,\ln K}\,i\,s}\, ds.
$$

## 3.2. Schéma d'Intégration Numérique

Pour évaluer numériquement l’intégrale

$$
\int_{0}^{+\infty}\frac{N(s)}{e^{i\,s\,\ln K}\,i\,s}\,ds,
$$

on procède de la manière suivante :

- On tronque l’intégrale à une borne supérieure fixée par exemple $s_{\max}=100$.
- L'intervalle $[0,100]$ est divisé en 1000 sous-intervalles de largeur :

  $$
  \Delta s \;=\; \frac{100}{1000} \;=\; 0.1.
  $$

- On applique la **méthode du point médian** pour chaque sous-intervalle en posant :

  $$
  s_j \;=\; \Delta s\left(j + \tfrac{1}{2}\right),\quad j=0,1,\dots,999.
  $$

- On somme ensuite les contributions de chacun des intervalles, puis on multiplie par $\frac{1}{\pi}$ et enfin on ajoute le terme analytique :

  $$
  \frac{1}{2}(S_t - K\,e^{-rT}).
  $$


---

# 4. Synthèse de la Démarche

1. **Fonction Caractéristique :**  
   La fonction $f_{\text{Heston}}(s)$ est construite soit en distinguant deux composantes $f_1(s)$ et $f_2(s)$, soit en les regroupant en une unique fonction caractéristique qui intègre les effets des paramètres $\kappa$, $\theta$, $\rho$, $\sigma$ et $v_0$ via des termes exponentiels complexes.

2. **Formule du Prix :**  
   Le prix de l’option s'exprime par :

   $$
   C(S_t,K,T) = \frac{1}{2}(S_t - K\,e^{-rT}) + \frac{1}{\pi}\int_{0}^{+\infty} \frac{N(s)}{e^{i\,s\,\ln K}\,i\,s}\,ds,
   $$

   avec

   $$
   N(s) = f_{\text{Heston}}(s - i) - K\,f_{\text{Heston}}(s).
   $$

3. **Approche Numérique :**  
   L'intégrale, initialement définie sur $[0,+\infty)$, est tronquée à $s_{\max}=100$ et évaluée à l’aide d’un schéma de Riemann par point médian sur 1000 intervalles.


In [ ]:
i = complex(0, 1)

def fHeston(
    s: complex,
    St: float,
    K: float,
    r: float,
    T: float,
    volvol: float,
    kappa: float,
    theta: float,
    v0: float,
    rho: float
) -> complex:
    prod = rho * volvol * i * s
    d1 = (prod - kappa) ** 2
    d2 = (volvol ** 2) * (i * s + s ** 2)
    d = np.sqrt(d1 + d2)

    g1 = kappa - prod - d
    g2 = kappa - prod + d
    g = g1 / g2

    exp1 = np.exp(np.log(St) * i * s) * np.exp(i * s * r * T)
    exp2 = 1 - g * np.exp(-d * T)
    exp3 = 1 - g
    mainExp1 = exp1 * np.power(exp2 / exp3, -2 * theta * kappa / (volvol ** 2))

    exp4 = theta * kappa * T / (volvol ** 2)
    exp5 = v0 / (volvol ** 2)
    exp6 = (1 - np.exp(-d * T)) / (1 - g * np.exp(-d * T))
    mainExp2 = np.exp(exp4 * g1 + exp5 * g1 * exp6)

    return mainExp1 * mainExp2


def priceHestonMid(
    St: float,
    K: float,
    r: float,
    T: float,
    volvol: float,
    kappa: float,
    theta: float,
    v0: float,
    rho: float
) -> float:
    P = 0.0
    iterations = 1000
    maxNumber = 100
    ds = maxNumber / iterations

    element1 = 0.5 * (St - K * np.exp(-r * T))

    for j in range(1, iterations):
        s1 = ds * (2 * j + 1) / 2
        s2 = s1 - i

        numerator1 = fHeston(s2, St, K, r, T, volvol, kappa, theta, v0, rho)
        numerator2 = K * fHeston(s1, St, K, r, T, volvol, kappa, theta, v0, rho)
        denominator = np.exp(np.log(K) * i * s1) * i * s1

        P += ds * (numerator1 - numerator2) / denominator

    element2 = P / np.pi
    return np.real(element1 + element2)



####  Objectif
Minimiser l’erreur quadratique pondérée entre les prix de marché (mid = (bid + ask)/2) et les prix donnés par le modèle de Heston, pour calibrer les paramètres `[kappa, theta, sigma, rho]`.

---

####  Choix des poids

Les poids sont définis comme l’inverse du carré de la volatilité implicite :

$$
w_i = \frac{1}{\sigma_{\text{imp},i}^2} \Bigg/ \sum_{j=1}^n \frac{1}{\sigma_{\text{imp},j}^2}
$$

Cela donne plus d’importance aux options faiblement volatiles (souvent plus liquides et précises), et réduit l’impact des options très excentrées ou bruitées.

---

####  Choix de ne pas calibrer `v₀`

La variance initiale `v₀` est déduite directement de la volatilité implicite de chaque option 


In [ ]:

def cost_function(
    params: Union[list, np.ndarray],
    data: pd.DataFrame,
    S0: float,
    market: "Market",
    weights: Optional[Union[np.ndarray, Sequence[float]]] = None
) -> float:
    """
    Least squares cost function for Heston model calibration.

    Parameters
    ----------
    params : list or np.ndarray
        Heston parameters [kappa, theta, sigma, rho].
    data : pd.DataFrame
        Market option data containing 'strike', 'time_to_maturity',
        'bid', 'ask', and 'implied_volatility' columns.
    S0 : float
        Spot price of the underlying asset.
    market : Market
        Market object providing get_rate(T) method to retrieve the risk-free rate.
    weights : array-like or None, optional
        Optional weighting scheme. If None, weights are derived from implied volatility.

    Returns
    -------
    float
        Weighted sum of squared errors between market mid-prices and model prices.
    """
    kappa, theta, sigma, rho = params

    # Extract market data
    strikes = data["strike"].values
    maturities = data["time_to_maturity"].values
    bids = data["bid"].values
    asks = data["ask"].values
    implied_vols = data["implied_volatility"].values
    v0 = implied_vols**2

    # Compute mid prices
    market_mid_prices = (bids + asks) / 2.0

    # Use weights if provided, otherwise use inverse-vol squared weights
    if weights is None:
        weights = 1 / (implied_vols**2)
        weights /= np.sum(weights)

    # Compute model prices
    model_prices = np.array([
        priceHestonMid(S0, k, market.get_rate(t), t, sigma, kappa, theta, v, rho)
        for k, t, v in zip(strikes, maturities, v0)
    ])

    # Compute weighted squared error
    errors = market_mid_prices - model_prices
    weighted_squared_errors = weights * errors**2
    return np.sum(weighted_squared_errors)



In [ ]:
from scipy.optimize import minimize, OptimizeResult
import pandas as pd
import numpy as np
from typing import List, Tuple, Union

def calibrate_heston(
    data: pd.DataFrame,
    S0: float,
    market: "Market",
    initial_guess: Union[List[float], np.ndarray],
    param_bounds: List[Tuple[float, float]]
) -> OptimizeResult:
    """
    Calibrate Heston model parameters using least squares minimization.

    Parameters
    ----------
    data : pd.DataFrame
        Market data with required columns ('strike', 'time_to_maturity', 'bid', 'ask', 'implied_volatility').
    S0 : float
        Spot price of the underlying asset.
    market : Market
        Market object with method get_rate(T).
    initial_guess : list or np.ndarray
        Initial guess for [kappa, theta, sigma, rho].
    param_bounds : list of tuples
        Bounds for each parameter in the form [(min, max), ...].

    Returns
    -------
    OptimizeResult
        Optimization result from scipy.optimize.minimize.
    """
    options = {
        'disp': True,
        'maxiter': 1000,
        'maxfun': 1000,
        'ftol': 1e-5,
        'gtol': 1e-5,
        'eps': 1e-5,
    }

    result = minimize(
        fun=lambda x: cost_function(x, data, S0, market),
        x0=initial_guess,
        bounds=param_bounds,
        method="L-BFGS-B",
        options=options
    )

    return result


Données AAPL déposées sur Phit Formation 

In [ ]:


file_path = "data/option_data/options_AAPL.csv"
df = pd.read_csv(file_path, sep=";")
df['price_date'] = pd.to_datetime(df['price_date'])
df['expiration'] = pd.to_datetime(df['expiration'])
df['time_to_maturity'] = (df['expiration'] - df['price_date']).dt.days / 365


On choisit les call ayant une volatilité raisonnable (1.3 de moneyness fixé arbitrairement)


In [ ]:
S0 = 216.9800  

df_test = df[(df['price_date'] == "2025-03-12")]

df_test = df_test[
    ((df_test['type'] == "call") & (df_test['strike'] >= S0 / 1.3))]  


In [26]:
security = "SPX"
rate_curve_type = RateCurveType.RF_US_TREASURY
interpolation_type = InterpolationType.SVENSSON
volatility_surface_type = VolatilitySurfaceType.SVI
day_count_convention = CalendarConvention.ACT_360
obs_frequency = ObservationFrequency.ANNUAL
market = Market(underlying_name=security,
                rate_curve_type=RateCurveType.RF_US_TREASURY,
                interpolation_type=InterpolationType.SVENSSON,
                volatility_surface_type=VolatilitySurfaceType.SVI,
                calendar_convention=CalendarConvention.ACT_360,
                obs_frequency=obs_frequency)

In [ ]:
S0 = 216.9800  

kappa = 2.0   # Mean reversion rate
theta = 0.05  # Long-term average volatility
sigma = 0.1   # Volatility of volatility
rho = -0.5    # Correlation coefficient

initial_guess = [kappa, theta, sigma,rho]  

param_bounds = [
        (1, 30.0),    # kappa
        (1e-3, 1.0),  # theta
        (1e-3, 1.0),  # sigma
        (-1.0, 1.0)]  # rho
 
res = calibrate_heston(df_test, S0, market, initial_guess, param_bounds)

In [146]:
kappa, theta, volvol,rho = res.x
print("Calibrated parameters:")
print(f"kappa: {kappa:.4f}, theta: {theta:.4f}, sigma: {volvol:.4f}, rho: {rho:.4f}")

Calibrated parameters:
kappa: 8.1471, theta: 0.0736, sigma: 0.3905, rho: -0.1707


In [143]:


kappa = 8.1471
theta = 0.0736
sigma = 0.3905
rho = -0.1707

maturities_to_plot = [1.021917808219178, 1.2684931506849315, 1.8465753424657534]

for maturity_target in maturities_to_plot:
    df_maturity = df_test[np.isclose(df_test['time_to_maturity'], maturity_target, atol=1e-3)]
    strikes = df_maturity['strike'].values
    maturities = df_maturity['time_to_maturity'].values
    market_prices = df_maturity['bid'].values
    implied_vols = df_maturity['implied_volatility'].values
    v0s = implied_vols**2

    model_prices = np.array([
        priceHestonMid(S0, k, market.get_rate(t), t, sigma, kappa, theta, v0, rho)
        for k, t, v0 in zip(strikes, maturities, v0s)
    ])

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=strikes, y=market_prices, mode='markers', name='Prix Marché'))
    fig.add_trace(go.Scatter(x=strikes, y=model_prices, mode='lines', name='Prix Modèle'))
    fig.update_layout(
        title=f'Comparaison des prix - Maturité ≈ {maturity_target:.3f} ans',
        xaxis_title='Strike',
        yaxis_title='Prix de l’option',
        template='plotly_white',
        legend=dict(x=0.05, y=0.95),
        width=800,
        height=500
    )
    fig.show()

